In [1]:
from probes import LRProbe
from utils import DataManager
import torch as th

In [5]:
labels = [
    'has_alice',
    'has_not',
    'label',
    'has_alice xor has_not',
    'has_alice xor label',
    'has_not xor label',
    'has_alice xor has_not xor label',
]
DEVICE = 'auto'
if DEVICE == 'auto':
    DEVICE = 'cuda' if th.cuda.is_available() else 'cpu'
model = 'EleutherAI/pythia-70m-deduped'
revision ="step6000"
accs = {}
for label in labels:
    dm = DataManager()
    for dataset in ['cities_alice', 'neg_cities_alice']:
        dm.add_dataset(dataset, model , 4, label=label, center=False, split=0.8, device=DEVICE, revision=revision)
    acts, labels = dm.get('train')
    probe = LRProbe.from_data(acts, labels, bias=True, device=DEVICE)
    acts, labels = dm.get('val')
    acc = (probe(acts).round() == labels).float().mean()
    accs[label] = acc

looking for activations in/home/clement/Documents/SPAR/geometry-of-truth/acts/EleutherAI/pythia-70m-deduped/step6000/cities_alice/layer_4_*.pt
looking for activations in/home/clement/Documents/SPAR/geometry-of-truth/acts/EleutherAI/pythia-70m-deduped/step6000/neg_cities_alice/layer_4_*.pt
looking for activations in/home/clement/Documents/SPAR/geometry-of-truth/acts/EleutherAI/pythia-70m-deduped/step6000/cities_alice/layer_4_*.pt
looking for activations in/home/clement/Documents/SPAR/geometry-of-truth/acts/EleutherAI/pythia-70m-deduped/step6000/neg_cities_alice/layer_4_*.pt
looking for activations in/home/clement/Documents/SPAR/geometry-of-truth/acts/EleutherAI/pythia-70m-deduped/step6000/cities_alice/layer_4_*.pt
looking for activations in/home/clement/Documents/SPAR/geometry-of-truth/acts/EleutherAI/pythia-70m-deduped/step6000/neg_cities_alice/layer_4_*.pt
looking for activations in/home/clement/Documents/SPAR/geometry-of-truth/acts/EleutherAI/pythia-70m-deduped/step6000/cities_alice/

In [6]:
import plotly.express as px
accs_f = {k: v.item() for k, v in accs.items()}
fig = px.bar(x=list(accs.keys()), y=list(accs_f.values()), text_auto=True)
fig.update_xaxes(title='Feature')
fig.update_yaxes(title='Accuracy')

In [25]:
dm = DataManager()
for dataset in ['cities']:
    dm.add_dataset(dataset, 'llama-2-13b', 14, label='label', center=False, split=0.8)
acts, labels = dm.get('train')
probe = LRProbe.from_data(acts, labels)
acts, labels = dm.get('val')
val_acc = (probe(acts).round() == labels).float().mean()

for dataset in ['neg_cities']:
    dm.add_dataset(dataset, 'llama-2-13b', 14, label='label', center=False, split=None)
acts, labels = dm.get('all')
test_acc = (probe(acts).round() == labels).float().mean()

px.bar(x=['On cities', 'On neg_cities'], y=[val_acc, test_acc], text_auto=True).update_xaxes(title='Dataset').update_yaxes(title='Accuracy')

In [4]:
dm = DataManager()
for dataset in ['cities_alice', 'neg_cities_alice']:
    dm.add_dataset(dataset, 'llama-2-13b', 14, label='has_alice', center=False, split=0.8)

In [5]:
acts, labels = dm.get('train')
probe = LRProbe.from_data(acts, labels)

/share/u/smarks/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
acts, labels = dm.get('val')
(probe(acts).round() == labels).float().mean()

tensor(1.)

In [109]:
dm = DataManager()
for dataset in ['cities_alice_only', 'neg_cities_alice_only']:
    dm.add_dataset(dataset, 'llama-2-13b', 14, label='xor', split=0.8)

In [110]:
acts, labels = dm.get('all')
(probe(acts).round() == labels).float().mean()

tensor(0.8068)

In [10]:
import pandas as pd

df = pd.read_csv('datasets/cities_alice.csv')
df['has_alice xor has_not xor label'] = df['has_alice'] ^ df['has_not'] ^ (df['label'] == 1)
df.to_csv('datasets/cities_alice.csv', index=False)

In [45]:
df.to_csv('datasets/neg_cities_distractor.csv', index=False)

In [119]:
df

,statement,label,city,country,correct_country,has_alice,has_not,has_alice xor has_not,has_alice xor label
0,Bob: The city of Krasnodar is in Russia.,1,Krasnodar,Russia,Russia,False,False,False,True
1,Alice: The city of Krasnodar is in South Africa.,0,Krasnodar,South Africa,Russia,True,False,True,True
2,Alice: The city of Lodz is in Poland.,1,Lodz,Poland,Poland,True,False,True,False
3,Bob: The city of Lodz is in the Dominican Repu...,0,Lodz,the Dominican Republic,Poland,False,False,False,False
4,Bob: The city of Maracay is in Venezuela.,1,Maracay,Venezuela,Venezuela,False,False,False,True
...,...,...,...,...,...,...,...,...,...
1491,Alice: The city of Kirkuk is in China.,0,Kirkuk,China,Iraq,True,False,True,True
1492,Bob: The city of Al Mawsil al Jadidah is in Iraq.,1,Al Mawsil al Jadidah,Iraq,Iraq,False,False,False,True
1493,Bob: The city of Al Mawsil al Jadidah is in In...,0,Al Mawsil al Jadidah,India,Iraq,False,False,False,False
1494,Bob: The city of Tangerang is in Indonesia.,1,Tangerang,Indonesia,Indonesia,False,False,False,True
